# astrometry
This tutorial shows you have to retrieve known astrometric parameters (RA, Dec, parallax, etc) for stellar and brown dwarfs stars.

## Initialize casparing

Import the required modules

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()

from casparing.derive.CASPAR_astrometry import multi_query_simbad, query_simbad, run_banyan, get_ages
from casparing.derive.CASPAR_data import add_astrometry_objects, save_astrometry_df
%load_ext autoreload
%autoreload 2

The Gaia archive will be intermittently unavailable from 08:00 to 09:00 on Monday, September 1st, as a result of scheduled maintenance (the deployment of a new version).


## Import object name(s)

You can add objects in four ways:
1. Filepath to a .csv, .txt. or .dat file.  The file should be one column with a header name. If the header name is not "obj", you must give the name, which will be renamed to "obj"


   > `
   >  objects = cdata.add_astrometry_objects(object_names='/path/to/csv/file.csv', colname='obj')
   >  `
2. Single object as a string

   > `
   > objects = cdata.add_astrometry_objects(object_names='CI Tau')
   > `
    
3. List or numpy array of objects
   > `
   > objects = cdata.add_astrometry_objects(object_names=['CI Tau', 'CQ Tau'])
   > `
4. Load object list as a pandas dataframe.  The dataframe should be one column with the header name "obj". If the header name is not "obj", you must give the name, which will be renamed to "obj" 
   >`
   >df = pd.read_csv('/path/to/csv/file.csv')
   >objects = cdata.add_astrometry_objects(object_names=df, header='obj')
   >`
   
NOTE: These objects should be SIMBAD resolvable names. 

In [2]:
# we give the example of using a list of name
objects = add_astrometry_objects(object_names=['CI Tau', 'CQ Tau'])
objects

,obj
0,CI Tau
1,CQ Tau


## Query simbad and Gaia to get RA, Dec, parallax, distance, pm for gaia dr2, and gaia edr3.

This can be done on a per object basis if you have one object.  If so, the step above (`cdata.add_astrometry_objects`) is not necessary.  

>`
>simbad_table = cmetry.query_simbad('CI Tau', verbose=True)
>`
>
or for all objects from `cdata.add_astrometry_objects` dataframe.

>`
>simbad_tables = df['obj'].progress_apply(cmetry.query_simbad)
>`

NOTE: This is relatively slow to run as it must run per object. 

In [3]:
# run query_simbad to get properties of all objects.  
# can also be used if you only want to query one object
simbad_df = multi_query_simbad(objects)
simbad_df

100%|█████████████████████████████████████████████| 2/2 [00:23<00:00, 11.54s/it]


,Reference Name,Simbad-Resolvable Name,RA (J2000.0),Dec (J2000.0),RA (J2016.0),Dec (J2016.0),GAIA DR2 Source ID,GAIA DR2 Parallax,GAIA DR2 Parallax err,GAIA DR2 Reliable Parallax,...,GAIA EDR3 Dec proper motion err,radial velocity,radial velocity err,Jmag,Jmag err,Hmag,Hmag err,Kmag,Kmag err,Links
0,CI Tau,2MASS J04335200+2250301,68.466727,22.841693,68.466770,22.841617,145203159127518336,6.300788,0.047268,True,...,0.019875,NaN,NaN,9.480,0.020,8.43,0.040,7.793,0.020,https://simbad.u-strasbg.fr/simbad/sim-id?Iden...
1,CQ Tau,2MASS J05355845+2444542,83.993611,24.748359,83.993626,24.748242,3416747570320423552,6.130522,0.082089,True,...,0.042136,31.404154,5.353594,7.926,0.023,7.06,0.024,6.173,0.018,https://simbad.u-strasbg.fr/simbad/sim-id?Iden...


## Get Association
We will now use BANYAN SIGMA to determine the association or star forming region for each object. This will add several columns to `simbad_df` in place: `Association`, `Association Probability Banyan Sigma` and `Association Census Reference`

In [4]:
run_banyan(simbad_df)
simbad_df

100%|█████████████████████████████████████████████| 2/2 [00:00<00:00,  4.19it/s]


,Reference Name,Simbad-Resolvable Name,RA (J2000.0),Dec (J2000.0),RA (J2016.0),Dec (J2016.0),GAIA DR2 Source ID,GAIA DR2 Parallax,GAIA DR2 Parallax err,GAIA DR2 Reliable Parallax,...,Jmag,Jmag err,Hmag,Hmag err,Kmag,Kmag err,Links,Association,Association Probability Banyan Sigma,Association Census Reference
0,CI Tau,2MASS J04335200+2250301,68.466727,22.841693,68.466770,22.841617,145203159127518336,6.300788,0.047268,True,...,9.480,0.020,8.43,0.040,7.793,0.020,https://simbad.u-strasbg.fr/simbad/sim-id?Iden...,Taurus,0.471641,Gange2018
1,CQ Tau,2MASS J05355845+2444542,83.993611,24.748359,83.993626,24.748242,3416747570320423552,6.130522,0.082089,True,...,7.926,0.023,7.06,0.024,6.173,0.018,https://simbad.u-strasbg.fr/simbad/sim-id?Iden...,FIELD,0.839378,Gange2018


## Determine Age
Using the association found above, we now use known cenuses of star forming regions to determine the association age of the objects.  This will add `Association Age` and `Association Age err` columns to `simbad_df` in place. For a complete list of the associations and ages, see Table 7 of Betti et al., (2023) or `data/parameters_data/Association Ages and Distances.csv`. 

In [5]:
get_ages(simbad_df)
simbad_df

2it [00:00, 96.09it/s]


,Reference Name,Simbad-Resolvable Name,RA (J2000.0),Dec (J2000.0),RA (J2016.0),Dec (J2016.0),GAIA DR2 Source ID,GAIA DR2 Parallax,GAIA DR2 Parallax err,GAIA DR2 Reliable Parallax,...,Hmag,Hmag err,Kmag,Kmag err,Links,Association,Association Probability Banyan Sigma,Association Census Reference,Association Age,Association Age err
0,CI Tau,2MASS J04335200+2250301,68.466727,22.841693,68.466770,22.841617,145203159127518336,6.300788,0.047268,True,...,8.43,0.040,7.793,0.020,https://simbad.u-strasbg.fr/simbad/sim-id?Iden...,Taurus/L1546,0.471641,Gange2018,2.01,0.32
1,CQ Tau,2MASS J05355845+2444542,83.993611,24.748359,83.993626,24.748242,3416747570320423552,6.130522,0.082089,True,...,7.06,0.024,6.173,0.018,https://simbad.u-strasbg.fr/simbad/sim-id?Iden...,FIELD,NaN,,NaN,NaN


## Save simbad table. 
We will finally use `cdata.save_astrometry_df` to save the dataframe. The parameters `savename` should indicate the full path and name to save the file as a .csv.  

In [6]:
save_astrometry_df(simbad_df, savename='/Users/sbetti22/Desktop/astrometry_table.csv')